# Atualizando documentos de um banco Mongo DB com o PyMongo

O PyMongo também nos permite inserir e atualizar documentos em um banco Mongo DB.

In [19]:
from pprint import pprint
from pymongo import MongoClient

client = MongoClient('localhost:27017') ## 27017 é a porta padrão para a instalação do Mongo DB
db = client.examples

### insert_one() e insert_many()

O método <b>insert_one()</b> nos permite inserir um novo documento em um banco de dados. O documento é passado como parâmetro. A inserção de múltiplos documentos é feita com <b>insert_many()</b>, que recebe uma array de documentos em JSON.

In [44]:
db.cities.insert_one({"name": "Natal"})
for doc in db.cities.find():
    pprint(doc)

{'_id': ObjectId('59ab956ce89f6f571f784127'), 'name': 'Natal'}


In [45]:
db.cities.insert_many([{"name": "Parnamirim"}, {"name": "Mossoró"}])
for doc in db.cities.find():
    pprint(doc)

{'_id': ObjectId('59ab956ce89f6f571f784127'), 'name': 'Natal'}
{'_id': ObjectId('59ab956de89f6f571f784128'), 'name': 'Parnamirim'}
{'_id': ObjectId('59ab956de89f6f571f784129'), 'name': 'Mossoró'}


### replace_one()

O método <b>replace_one()</b> nos permite substituir o conteúdo de um documento que responda a um determinada filtro.
```
replace_one(filtro,document_replacement)
```

In [46]:
db.cities.replace_one({"name": "Natal"}, {"name": "Natal", "state":"RN"})
for doc in db.cities.find():
    pprint(doc)

{'_id': ObjectId('59ab956ce89f6f571f784127'), 'name': 'Natal', 'state': 'RN'}
{'_id': ObjectId('59ab956de89f6f571f784128'), 'name': 'Parnamirim'}
{'_id': ObjectId('59ab956de89f6f571f784129'), 'name': 'Mossoró'}


Se tivermos em mente o exato documento que desejamos substituir, podemos usar o campo "_id" como filtro.

In [47]:
from bson.objectid import ObjectId

db.cities.replace_one({"_id":ObjectId('59ab89dce89f6f571f784124')}, {"name": "Parnamirim", "state":"RN"})
for doc in db.cities.find():
    pprint(doc)

{'_id': ObjectId('59ab956ce89f6f571f784127'), 'name': 'Natal', 'state': 'RN'}
{'_id': ObjectId('59ab956de89f6f571f784128'), 'name': 'Parnamirim'}
{'_id': ObjectId('59ab956de89f6f571f784129'), 'name': 'Mossoró'}


Quando ativamos a opção <b>upsert</b>, setada como False por padrão, o comando irá inserir um novo documento caso não encontre nenhum resultado que responda ao filtro especificado.

In [48]:
db.cities.replace_one({"name":"Caicó"}, {"name": "Caicó", "state":"RN"})
for doc in db.cities.find():
    pprint(doc)

{'_id': ObjectId('59ab956ce89f6f571f784127'), 'name': 'Natal', 'state': 'RN'}
{'_id': ObjectId('59ab956de89f6f571f784128'), 'name': 'Parnamirim'}
{'_id': ObjectId('59ab956de89f6f571f784129'), 'name': 'Mossoró'}


In [49]:
db.cities.replace_one({"name":"Caicó"}, {"name": "Caicó", "state":"RN"}, upsert=True)
for doc in db.cities.find():
    pprint(doc)

{'_id': ObjectId('59ab956ce89f6f571f784127'), 'name': 'Natal', 'state': 'RN'}
{'_id': ObjectId('59ab956de89f6f571f784128'), 'name': 'Parnamirim'}
{'_id': ObjectId('59ab956de89f6f571f784129'), 'name': 'Mossoró'}
{'_id': ObjectId('59ab95911ae523d3f3c8c9bd'), 'name': 'Caicó', 'state': 'RN'}


### update_one()

<b>update_one()</b> nos permite atualizar o conteúdo de um único documento que responda a um determinad filtro. A principal diferença em relação ao replace_one() é que, quando usado em conjunto com o operador <b>\$set</b>, o conteúdo do documento original não será completamente substituído pelo que consta no segundo parâmetro. Ao invés disso, os campos especificados serão atualizados e, caso algum deles não exista, será criado, mantendo os campos já existentes no documento original.

```
replace_one(filtro,document_update)
```

In [52]:
db.cities.update_one({"name":"Caicó"}, {"$set":{"state":"Rio Grande do Norte", "country":"Brasil"}})
for doc in db.cities.find():
    pprint(doc)

{'_id': ObjectId('59ab956ce89f6f571f784127'), 'name': 'Natal', 'state': 'RN'}
{'_id': ObjectId('59ab956de89f6f571f784128'), 'name': 'Parnamirim'}
{'_id': ObjectId('59ab956de89f6f571f784129'), 'name': 'Mossoró'}
{'_id': ObjectId('59ab95911ae523d3f3c8c9bd'),
 'country': 'Brasil',
 'name': 'Caicó',
 'state': 'Rio Grande do Norte'}


Além do \$set, outros operadores de atualização de campos podem ser utilizados. Uma lista com todos os operadores disponíveis pode ser encontrada [aqui](https://docs.mongodb.com/manual/reference/operator/update-field/).

### update_many()

<b>update_many()</b> nos permite atualizar todos os documentos que correspondam ao filtro especificado. Esse método também possui uma opção de <b>upsert</b>, desativada por padrão, que nos permite inserir um novo documento se o filtro especificado der resposta vazia.

In [53]:
db.cities.update_many({"name":{"$exists":1}}, {"$set":{"state":"Rio Grande do Norte", "country":"Brasil"}})
for doc in db.cities.find():
    pprint(doc)

{'_id': ObjectId('59ab956ce89f6f571f784127'),
 'country': 'Brasil',
 'name': 'Natal',
 'state': 'Rio Grande do Norte'}
{'_id': ObjectId('59ab956de89f6f571f784128'),
 'country': 'Brasil',
 'name': 'Parnamirim',
 'state': 'Rio Grande do Norte'}
{'_id': ObjectId('59ab956de89f6f571f784129'),
 'country': 'Brasil',
 'name': 'Mossoró',
 'state': 'Rio Grande do Norte'}
{'_id': ObjectId('59ab95911ae523d3f3c8c9bd'),
 'country': 'Brasil',
 'name': 'Caicó',
 'state': 'Rio Grande do Norte'}


update_one() e update_many() também possuem uma opção <b>upsert</b>, desativada por padrão, que, se ativada, fará com que os comandos insiram um novo documento caso o filtro especificado tenha resposta vazia.

### delete_one() e delete_many()

O método <b>delete_one()</b> nos permite deletar o primeiro documento que corresponda ao filtro especificado. <b>delete_many()</b> irá deletar todos os documentos que correspondam ao filtro que foi passado como parâmetro.

In [58]:
db.cities.insert_many([{"name": "Macaíba"},{"name":"São Gonçalo"},{"name":"Ceará Mirim"}])

In [59]:
db.cities.delete_one({"state":{"$exists":0}})
for doc in db.cities.find():
    pprint(doc)

{'_id': ObjectId('59ab956ce89f6f571f784127'),
 'country': 'Brasil',
 'name': 'Natal',
 'state': 'Rio Grande do Norte'}
{'_id': ObjectId('59ab956de89f6f571f784128'),
 'country': 'Brasil',
 'name': 'Parnamirim',
 'state': 'Rio Grande do Norte'}
{'_id': ObjectId('59ab956de89f6f571f784129'),
 'country': 'Brasil',
 'name': 'Mossoró',
 'state': 'Rio Grande do Norte'}
{'_id': ObjectId('59ab95911ae523d3f3c8c9bd'),
 'country': 'Brasil',
 'name': 'Caicó',
 'state': 'Rio Grande do Norte'}
{'_id': ObjectId('59ab98bce89f6f571f78412b'), 'name': 'São Gonçalo'}
{'_id': ObjectId('59ab98bce89f6f571f78412c'), 'name': 'Ceará Mirim'}


In [60]:
db.cities.delete_many({"state":{"$exists":0}})
for doc in db.cities.find():
    pprint(doc)

{'_id': ObjectId('59ab956ce89f6f571f784127'),
 'country': 'Brasil',
 'name': 'Natal',
 'state': 'Rio Grande do Norte'}
{'_id': ObjectId('59ab956de89f6f571f784128'),
 'country': 'Brasil',
 'name': 'Parnamirim',
 'state': 'Rio Grande do Norte'}
{'_id': ObjectId('59ab956de89f6f571f784129'),
 'country': 'Brasil',
 'name': 'Mossoró',
 'state': 'Rio Grande do Norte'}
{'_id': ObjectId('59ab95911ae523d3f3c8c9bd'),
 'country': 'Brasil',
 'name': 'Caicó',
 'state': 'Rio Grande do Norte'}


### drop() e drop_collection

<b>drop() e drop_collection()</b> permitem deletar uma coleção por completo.

In [42]:
#db.cities.drop()
#db.drop_collection("cities")